In [2]:
import wrds
import pandas as pd
import numpy as np

# ============================================================
# 0. Connexion WRDS
# ============================================================

db = wrds.Connection()

# ============================================================
# 1. Tickers S&P 500 présents TOUTE la période 2003-2023
#    via CRSP (actions ordinaires seulement)
# ============================================================

sp500_crsp = db.raw_sql("""
    WITH full_members AS (
        SELECT
            permno,
            MIN(start) AS min_start,
            MAX(COALESCE(ending, '9999-12-31')) AS max_end
        FROM crsp.msp500list
        GROUP BY permno
        HAVING MIN(start) <= '2003-01-01'
           AND MAX(COALESCE(ending, '9999-12-31')) >= '2023-12-31'
    )
    SELECT DISTINCT
        c.permno,
        c.tsymbol AS tic
    FROM full_members AS f
    JOIN crsp.msenames AS c
      ON f.permno = c.permno
    WHERE c.tsymbol IS NOT NULL
      AND c.shrcd IN (10, 11)    -- seulement common shares
      AND c.namedt <= '2023-12-31'
      AND (c.nameendt >= '2003-01-01' OR c.nameendt IS NULL)
""")

sp500_crsp["tic"] = sp500_crsp["tic"].astype(str).str.strip()
sp500_tickers = sp500_crsp["tic"].drop_duplicates()

print("Tickers S&P500 (présents toute la période 2003-2023) :", len(sp500_tickers))

# ============================================================
# 2. Secteurs Compustat via comp.secm + comp.company
#    - comp.secm : gvkey, tic
#    - comp.company : gvkey, gsector, gsubind
# ============================================================

comp_secm = db.raw_sql("""
    SELECT DISTINCT
        gvkey,
        tic
    FROM comp.secm
    WHERE tic IS NOT NULL
""")
comp_secm["tic"] = comp_secm["tic"].astype(str).str.strip()
comp_secm["gvkey"] = comp_secm["gvkey"].astype(str).str.strip()

comp_company = db.raw_sql("""
    SELECT
        gvkey,
        gsector,
        gsubind
    FROM comp.company
    WHERE gsector IS NOT NULL
""")
comp_company["gvkey"] = comp_company["gvkey"].astype(str).str.strip()
comp_company["gsector"] = comp_company["gsector"].astype(str).str.strip()
comp_company["gsubind"] = comp_company["gsubind"].astype(str).str.strip()

# Merge pour avoir (tic, gsector, gsubind)
comp_tic_sector = comp_secm.merge(comp_company, on="gvkey", how="left")

print("Lignes comp_tic_sector :", len(comp_tic_sector))

# ============================================================
# 3. Merge S&P500 (CRSP) avec les secteurs Compustat
# ============================================================

merged = sp500_crsp.merge(
    comp_tic_sector[["tic", "gsector", "gsubind"]],
    on="tic",
    how="left"
)

print("Après merge S&P500 x secteurs :", len(merged))

# ============================================================
# 4. Filtre secteurs fin : exclure
#    - Real Estate (gsector=60)
#    - Oil / Coal : gsubind ~ 10102*
#    - Mines / Gold / Steel / Metals & Mining : gsubind ~ 15104*
# ============================================================

gsector = merged["gsector"].astype(str)
gsubind = merged["gsubind"].astype(str)

# Real Estate = tout gsector commençant par "60"
mask_real_estate = gsector.str.startswith("60")

# Oil, Gas & Consumable Fuels (inclut coal) -> 10102xx
# Metals & Mining / Gold / Steel -> 15104xx
exclude_gsubind_prefixes = ("10102", "15104")
mask_oil_mines_gold_steel = gsubind.str.startswith(exclude_gsubind_prefixes)

mask_exclu = mask_real_estate | mask_oil_mines_gold_steel

merged_filtered = merged[~mask_exclu].copy()

print("Tickers exclus (oil/coal/mines/gold/steel/RE) :", mask_exclu.sum())
print("Tickers restants après filtrage               :", merged_filtered["tic"].nunique())

tickers_filtered = merged_filtered["tic"].drop_duplicates()

# ============================================================
# 5. Tirage aléatoire de 50 tickers parmi ceux restants
# ============================================================

np.random.seed(42)
n_tickers = min(50, len(tickers_filtered))
tickers_50 = np.random.choice(tickers_filtered, size=n_tickers, replace=False)

print("Tickers sélectionnés :", tickers_50)

tickers_sql = "'" + "', '".join(tickers_50) + "'"

# ============================================================
# 6. Données journalières Compustat (secd) pour ces tickers
# ============================================================

query = f"""
    SELECT
        s.tic,
        s.datadate,
        s.eps,
        s.prccd,
        s.gvkey
    FROM comp_na_daily_all.secd AS s
    WHERE s.datadate BETWEEN '2003-01-01' AND '2023-12-31'
      AND s.tic IN ({tickers_sql})
"""

df = db.raw_sql(query)

df["datadate"] = pd.to_datetime(df["datadate"])
df = df.sort_values(["tic", "datadate"])

# ============================================================
# 7. Nettoyage numérique + P/E instantané
# ============================================================

df["eps"]   = pd.to_numeric(df["eps"], errors="coerce")
df["prccd"] = pd.to_numeric(df["prccd"], errors="coerce")

df["pe_ratio"] = np.where(
    (df["eps"].notna()) & (df["eps"] > 0),
    df["prccd"] / df["eps"],
    np.nan
)

# ============================================================
# 8. Export Excel
# ============================================================

output_file = "sp500_filtered_random50.xlsx"
df.to_excel(output_file, index=False)

print("\nFichier Excel créé :", output_file)


WRDS recommends setting up a .pgpass file.
pgpass file created at C:\Users\berge\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Tickers S&P500 (présents toute la période 2003-2023) : 273
Lignes comp_tic_sector : 61550
Après merge S&P500 x secteurs : 273
Tickers exclus (oil/coal/mines/gold/steel/RE) : 20
Tickers restants après filtrage               : 253
Tickers sélectionnés : ['PARA' 'FISV' 'AEE' 'PMTCE' 'VFC' 'HD' 'EA' 'SCHW' 'DTE' 'MHP' 'CMCSA'
 'EMN' 'HIG' 'ABC' 'GWW' 'TXT' 'SCH' 'IBM' 'AHC' 'EIX' 'ADSK' 'AMGN' 'MMC'
 'TAP' 'RTX' 'JPM' 'PRU' 'CSCO' 'ETN' 'VZ' 'CI' 'BFB' 'NTRS' 'ATH' 'COF'
 'HSY' 'DOV' 'MO' 'FPL' 'HAS' 'BHGE' 'LTD' 'EBAY' 'FDX' 'ABT' 'CINF' 'PCG'
 'WEC' 'PTC' 'ADBE']

Fichier Excel créé : sp500_filtered_random50.xlsx
